In [245]:
#Import Packages
from sklearn.datasets import load_files
from glob import glob
#from scipy.misc import imread
from sklearn.model_selection import train_test_split

import numpy as np
import time
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

In [269]:
FIXED_IMG_SIZE = 227

#Load the full dataset, scale the images in values between 0-1 and resize to size 227
def load_dataset(path, target):
    filenames = glob(path)
    
    if target == 0:
        cls = [1,0] #female - 0
    else:
        cls = [0,1] #male - 1
        
    target_array = np.full(shape=(len(filenames),2), fill_value=cls, dtype='float32')
    #target_array = np.ndarray((len(filenames),1), target)
    
    images = [cv2.imread(file) for file in filenames]
    images = [cv2.resize(img, (FIXED_IMG_SIZE, FIXED_IMG_SIZE)) for img in images]
    
    images = np.array(images, dtype = np.uint8)
    images = images.astype('float32')
    images = np.multiply(images, 1.0/255.0)
    
    train_x, test_x, train_y, test_y = train_test_split(images.tolist(), target_array, test_size=0.4) #solve problem
    valid_x, test_x, valid_y, test_y = train_test_split(test_x, test_y, test_size=0.5)
        
    return train_x, valid_x, test_x, train_y, valid_y, test_y

In [270]:
print('Loading Datasets')
fem_train_x, fem_valid_x, fem_test_x, fem_train_y, fem_valid_y, fem_test_y = female_test = load_dataset('data/female1/*', target=1)
print('Loaded Female Dataset')

male_train_x, male_valid_x, male_test_x, male_train_y, male_valid_y, male_test_y = load_dataset('data/male1/*', target=0)
print('Loaded Male Dataset')

train_len = len(fem_train_x) + len(male_train_x)
valid_len = len(fem_train_x) + len(male_valid_x)
test_len = len(fem_test_x) + len(male_test_x)

print('Both datasets loaded')
print('Number of Training Data: ' + str(train_len))
print('Number of Validation Data: ' + str(valid_len))
print('Number of Testing Data: ' + str(test_len))

Loading Datasets
Loaded Female Dataset
Loaded Male Dataset
Both datasets loaded
Number of Training Data: 48
Number of Validation Data: 32
Number of Testing Data: 16


In [271]:
#Inputs
train_x = np.asarray(fem_train_x + male_train_x)
valid_x = np.asarray(fem_valid_x + male_valid_x)
test_x = np.asarray(fem_test_x + male_test_x)

#targets 
train_y = np.concatenate((fem_train_y, male_train_y), axis=0)
valid_y = np.concatenate((fem_valid_y, male_valid_y), axis=0)
test_y = np.concatenate((fem_test_y, male_test_y), axis=0)

In [272]:
import tensorflow as tf
def create_weights(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.01), name='create_weights')

def create_biases(num_filters):
    return tf.Variable(tf.zeros(num_filters), name='create_biases') #CHECK WITH NON-ZERO VALUE                       

In [273]:
def create_conv_layer(input, num_input_channels, filter_size, num_filters, conv_strides=1, conv_padding='SAME'):
    #define weights and biases
    weights = create_weights(shape=[filter_size, filter_size, num_input_channels, num_filters])
    print(weights.get_shape())
    biases = create_biases(num_filters=num_filters)
    
    #define conv2d layer
    conv_layer = tf.nn.conv2d(input=input, filter=weights, strides=[1,conv_strides,conv_strides,1], padding=conv_padding)
    conv_layer += biases
    
    #max pooling layer
    conv_layer = tf.nn.max_pool(value=conv_layer, ksize=[1,2,2,1], padding='SAME', strides=[1,2,2,1])
    
    #relu layer
    conv_layer = tf.nn.relu(conv_layer)
    
    #local response normalization layer
    conv_layer = tf.nn.local_response_normalization(conv_layer)
    
    return conv_layer

In [274]:
#flatten layer
def create_flatten_layer(layer):
    layer_shape = layer.get_shape()
    num_elements = layer_shape[1:4].num_elements()
    layer = tf.reshape(layer, [-1, num_elements])
    
    return layer

In [275]:
#create fully connected layers
def create_fully_conn_layer(input, num_inputs, num_outputs, use_relu=True, use_dropout=True):
    #define weights and biases for fully connected layer
    weights = create_weights(shape=[num_inputs, num_outputs])
    biases = create_biases(num_outputs)
    
    fconn_layer = tf.matmul(input, weights) + biases
    if use_relu:
        fconn_layer = tf.nn.relu(fconn_layer)
        
    if use_dropout:
        fconn_layer = tf.nn.dropout(fconn_layer, keep_prob=0.2)
        
    return fconn_layer

In [ ]:
# def next_batch(batch_size, data, labels): 
    
#     num_batches = data.shape[0] // batch_size
#     print(data.shape[0])
    
#     data = data[:num_batches*batch_size]
#     labels = labels[:num_batches*batch_size]
    
#     count = 1
#     for i in range(0, data.shape[0], batch_size):
        
#         if count > num_batches:
#             count += 1
#             i = 0
        
#         x = data[i : i + batch_size ,:]
#         y = np.squeeze(labels[i : i + batch_size ,:], axis=1)
        
#         count += 1
#         yield x, y

In [277]:
def shuffle_dataset_batch(batch_size, data, labels):
    idx = np.arange(0, len(data))
    np.random.shuffle(idx)
    idx = idx[:batch_size]
    data_shuffle = [data[i] for i in idx]
    label_shuffle = [labels[i] for i in idx]
    
    return np.asarray(data_shuffle), np.asarray(label_shuffle)

In [285]:
#Global Variables
batch_size = 16
num_epochs = 10
learning_rate=1e-4

#define placeholders and input
num_channels = 3
num_classes = 2
x = tf.placeholder(dtype=tf.float32, shape=[batch_size, FIXED_IMG_SIZE, FIXED_IMG_SIZE, num_channels], name='input_x') 
y_true = tf.placeholder(dtype=tf.float32, shape=[batch_size, num_classes], name='y_true')
y_true_cls = tf.argmax(y_true, dimension=1)

In [286]:
#Run Tensorflow session
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    #design the cnn for the problem statement
    print(input_x.get_shape())

    layer_conv1 = create_conv_layer(input=input_x, num_filters = 96, filter_size=7, conv_strides=4, conv_padding='VALID', num_input_channels=num_channels)
    print(layer_conv1.get_shape())

    layer_conv2 = create_conv_layer(input=layer_conv1, num_filters = 256, filter_size=5, conv_strides=1, 
                                    conv_padding='SAME', num_input_channels=96)
    print(layer_conv2.get_shape())

    layer_conv3 = create_conv_layer(input=layer_conv2, num_filters = 256, filter_size=3, conv_strides=1, 
                                    conv_padding='SAME', num_input_channels=256)
    print(layer_conv3.get_shape())

    layer_flat = create_flatten_layer(layer=layer_conv3)
    print(layer_flat.get_shape())

    layer_fc1 = create_fully_conn_layer(input=layer_flat, num_inputs=layer_flat.get_shape()[1:4].num_elements(),
                                       num_outputs=512)
    print(layer_fc1.get_shape())

    layer_fc2 = create_fully_conn_layer(input=layer_fc1, num_inputs=512, num_outputs=512)
    print(layer_fc2.get_shape())

    layer_fc3 = create_fully_conn_layer(input=layer_fc2, num_inputs=512, num_outputs=num_classes, 
                                        use_relu=False, use_dropout=False)
    print(layer_fc3.get_shape())

    #softmax layer
    y_pred = tf.nn.softmax(layer_fc3, name='y_pred')
    y_pred_cls = tf.argmax(y_pred, dimension=1)

    # #define cost
    loss = tf.nn.softmax_cross_entropy_with_logits(logits=layer_fc3, labels=y_true)
    cost = tf.reduce_mean(loss)

    #adam optimisation
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost) #learning rate variable
    
    for e in range(1, num_epochs):

        start_time = time.time()

        x_train_batch, y_train_batch = shuffle_dataset_batch(batch_size, data=train_x, labels=train_y)
        x_valid_batch, y_valid_batch = shuffle_dataset_batch(batch_size, data=valid_x, labels=valid_y)

        feed_dict_train = {x : x_train_batch, y_true : y_train_batch}
        feed_dict_valid = {x : x_valid_batch, y_true : y_valid_batch}
        print(sess.run(optimizer, feed_dict=feed_dict_train))

(?, 227, 227, 3)
(7, 7, 3, 96)
(?, 28, 28, 96)
(5, 5, 96, 256)
(?, 14, 14, 256)
(3, 3, 256, 256)
(?, 7, 7, 256)
(?, 12544)
(?, 512)
(?, 512)
(?, 2)


InvalidArgumentError: You must feed a value for placeholder tensor 'input_2' with dtype float
	 [[Node: input_2 = Placeholder[dtype=DT_FLOAT, shape=[], _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'input_2', defined at:
  File "/Users/veronica/anaconda3/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/veronica/anaconda3/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/veronica/anaconda3/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/veronica/anaconda3/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/veronica/anaconda3/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/Users/veronica/anaconda3/anaconda3/lib/python3.6/si

In [238]:
y_train_batch.shape

(16,)

In [214]:
train_y.shape

(48, 1)

In [12]:
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_alt.xml')

In [13]:
#detect human face boundaries
def face_detector(img_path):
    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray)
    
    return len(faces) > 0